In [4]:
import os
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFDirectoryLoader


groq_api_key=os.getenv('GROQ_API_KEY')
llm=ChatGroq(groq_api_key=groq_api_key,model_name="llama-3.3-70b-versatile",temperature=0.3)

response=llm.invoke("what is fax? How is it different from efax?")
response.content

"**What is Fax?**\n\nFax (short for facsimile) is a method of transmitting printed or written documents over a telephone line. It was widely used in the 1980s and 1990s for sending and receiving documents, especially in business and professional settings. The process involves scanning a physical document and converting it into a digital signal, which is then transmitted over a phone line to a receiving fax machine. The receiving machine prints out a copy of the original document.\n\nHere's how traditional faxing works:\n\n1. A document is placed in a fax machine.\n2. The fax machine scans the document and converts it into a digital signal.\n3. The digital signal is transmitted over a phone line to the receiving fax machine.\n4. The receiving fax machine receives the signal and prints out a copy of the original document.\n\n**What is eFax?**\n\neFax (short for electronic fax) is a digital alternative to traditional faxing. It allows users to send and receive faxes over the internet, eli

In [5]:
response=llm.invoke("What are the components of soil?")

lines=response.content.split("\n")
for line in lines:
    print(line)

Soil is a complex and dynamic ecosystem composed of several key components. The main components of soil are:

1. **Mineral particles**: These include sand, silt, and clay particles, which provide the soil's structure and texture. The proportions of these particles determine the soil's classification (e.g., clay, loam, sandy).
2. **Organic matter**: This includes decomposed plant and animal residues, such as humus, which provides nutrients, improves soil structure, and supports soil biota.
3. **Water**: Soil contains varying amounts of water, which is essential for plant growth and microbial activity.
4. **Air**: Soil contains pore spaces that allow for air exchange, which is necessary for root growth, microbial respiration, and other biological processes.
5. **Living organisms**: Soil is home to a diverse array of living organisms, including:
	* **Microorganisms** (bacteria, fungi, protozoa): These microbes play crucial roles in decomposing organic matter, fixing nitrogen, and solubili

# Prompt template

In [16]:
from langchain.prompts import PromptTemplate

noinputprompt=PromptTemplate(input_variables=[],template="Tell me a python trick")
print(noinputprompt)

input_variables=[] input_types={} partial_variables={} template='Tell me a python trick'


In [18]:
formattednoinputprompt=noinputprompt.format()
response=llm.invoke(formattednoinputprompt)

lines=response.content.split("\n")
for line in lines:
    print(line)

<think>
Okay, so I just learned this Python trick about using the `enumerate` function to loop over both the index and the value of each item in a list. I think it's pretty cool because before, I used to use `range(len(list))` to get the index and then access the element with `list[i]`, which seems a bit cumbersome now that I know `enumerate`.

Let me try to break this down. Normally, when I loop through a list, I might do something like:

for item in list:
    print(item)

But that only gives me the item, not the index. So if I need both, I might have used:

for i in range(len(list)):
    print(i, list[i])

But that's more code and a bit less readable. With `enumerate`, I can get both the index and the item in a cleaner way. The example given was:

fruits = ['apple', 'banana', 'cherry']
for index, fruit in enumerate(fruits):
    print(f"Fruit number {index}: {fruit}")

This outputs each fruit with its index, starting from 0. That makes sense.

Wait, but what if I want the index to sta

In [6]:
noinputprompt=PromptTemplate.from_template(template="Tell me a python trick")
print(noinputprompt)

input_variables=[] template='Tell me a python trick'


### Single input prompt template

In [45]:
import re

oneinput = PromptTemplate(
    input_variables=["keyword"],
    template="""As a professional SEO strategist, generate 3 high-potential keywords for {keyword} following these STRICT guidelines:
1. Output MUST be 3 lines, each containing ONLY one keyword.
2. Keywords must include commercial intent modifiers like "best," "near me," "cost," or "vs."
3. Keywords must be 2-5 words long.
4. Exclude informational terms like "what," "how," or "why."
5. Use Title Case formatting.
6. Ensure keywords are actionable and rankable.

BAD EXAMPLE (Avoid):
- Doctor Qualifications
- Medical Practitioner Licensing
- Physician Education Requirements

GOOD EXAMPLE:
Best Cardiologist Near Me
Pediatrician Vs Family Doctor Costs
24/7 Emergency Doctors [City]

Generate COMPETITIVE keywords for: {keyword}"""
)

# User input (dynamic)
user_keyword = "fax cover sheet"  # Replace with actual user input
formatoneinput = oneinput.format(keyword=user_keyword)
response = llm.invoke(formatoneinput)

# Process the response
keyword_list = []
for line in response.content.split("\n"):
    # Clean the line
    cleaned = re.sub(r'[^a-zA-Z0-9\s\-\&]', '', line.strip())  # Remove special chars
    # Skip empty lines or lines that don't meet criteria
    if cleaned and 2 <= len(cleaned.split()) <= 5:
        keyword_list.append(cleaned.title())

keyword_list

['Best Fax Cover Sheet',
 'Free Fax Cover Sheet Template',
 'Fax Cover Sheet Vs Letterhead']

In [21]:
oneinput=PromptTemplate.from_template("Tell me a the capital of the {country}")
formattedoneinput=oneinput.format(country="Russia")
response=llm.invoke(formattedoneinput)
lines=response.content.split("\n")
for line in lines:
    print(line)

<think>
Okay, so I need to find out the capital of Russia. Hmm, I think I remember that Russia is a big country, maybe the largest in the world. I'm pretty sure the capital isn't somewhere like Moscow or Saint Petersburg. Wait, no, I think Moscow is the capital. But let me think about this more carefully.

First, I know that capitals are the main cities where the government is located. So, for Russia, I believe it's Moscow. But I'm not 100% sure. I've heard Saint Petersburg mentioned a lot too. Maybe I should recall some history. I think in the past, Saint Petersburg was the capital, but that changed. When was that? Maybe during the Soviet era or earlier?

I remember that Saint Petersburg was founded by Peter the Great and was the capital for a long time. Then, I think the capital moved to Moscow, perhaps around the time of the Russian Revolution or maybe earlier. I'm not exactly sure about the timeline. So, maybe Moscow has been the capital for a while now.

Another way to think about

### Multiple input prompt template

In [22]:
multiinput=PromptTemplate(input_variables=["country","anything"],template="Can you tell me about the {country} and its famous {anything}")
formattedmulti=multiinput.format(country="India",anything="cities")
response=llm.invoke(formattedmulti)
lines=response.content.split("\n")
for line in lines:
    print(line)

<think>
Okay, so I need to learn about India and its famous cities. I know India is a big country in South Asia, but I'm not exactly sure where all. I think it's near Pakistan and Bangladesh? Maybe Nepal too? I remember hearing about the Taj Mahal, which is in Agra, right? So Agra must be one of the cities.

I know Delhi is the capital, and it's a really crowded place. There's also Mumbai, which I think is the financial hub, like their version of New York. I've heard of Bollywood movies coming from Mumbai. Then there's Bangalore, which I believe is called Bengaluru now. I think it's known for technology and IT companies, like India's Silicon Valley.

Kolkata was called Calcutta before, I think. It's in the east, maybe near Bangladesh. I'm not too sure about what it's famous for, but I think it has a rich history. Chennai is another city I've heard of, maybe in the south? It might be a port city or something. Hyderabad is another one, known for IT as well, I think. Oh, and Pune is near 

In [23]:
multiinput=PromptTemplate.from_template("Tell me a {language}{topic} trick")
formattedmulti=multiinput.format(language="python",topic="array")
response=llm.invoke(formattedmulti)
lines=response.content.split("\n")
for line in lines:
    print(line)

<think>
Okay, so I'm trying to figure out how to use the `array` module in Python for manipulating arrays efficiently. I remember the user mentioned that it's a powerful module with various data types, but I'm not entirely sure how to use it properly. Let me start by recalling what I know about Python arrays.

I know that Python has a built-in list type, which is versatile, but I've heard that the `array` module offers more efficient storage for homogeneous data. So, if I have a lot of numbers of the same type, using an array might save memory and improve performance. But how exactly do I create and use an array?

First, I think I need to import the `array` module. So I'll start with `import array`. Then, I believe there are different data types you can specify when creating an array. The user mentioned letters like 'i' for signed integers, 'f' for floats, etc. So maybe when I create an array, I have to specify the typecode.

Let me try to create an integer array. I think the syntax is

# Chat Models Prompt Template

In [24]:
from langchain_core.prompts import ChatPromptTemplate,SystemMessagePromptTemplate,HumanMessagePromptTemplate,PromptTemplate

In [3]:
chatprompt=ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant that translates {input_language} to {output_language}"),
    ("human","{text}")
])

formatchatprompt=chatprompt.format_messages(
    input_language="English",
    output_language="Kannada",
    text="Hello, how are you nice to meet you"
)

print(formatchatprompt)

[SystemMessage(content='You are a helpful assistant that translates English to Kannada'), HumanMessage(content='Hello, how are you nice to meet you')]


In [27]:
res=llm.invoke(formatchatprompt)
res.content

"ನಿಮ್ಮನ್ನು ಭೇಟಿ ಮಾಡಿದ್ದೇನೆ ಸ್ವಾಗತ! Namaste! (It's nice to meet you!)"

In [37]:
chatprompt=ChatPromptTemplate.from_messages([
    ("system","You are a creative and funny chat assistant that provides captions for social media posts based on three words that describe the post {word1},{word2},{word3}"),
])

formatchatprompt=chatprompt.format_messages(
    word1="happy",
    word2="twinning",
    word3="beach",
) 

res=llm.invoke(formatchatprompt)
lines=res.content.split('\n')
for line in lines:
    print(line)

"Sand, Sun, and Sisterly Love! Twins' Beach Day #Twinning #BeachVibes #SisterGoals"


In [41]:
chatprompt=ChatPromptTemplate.from_messages([
    ("system","Imagine a witty and inventive chat assistant that crafts social media captions tailored to posts, using three descriptive words: {word1}, {word2}, and {word3}."),
])

formatchatprompt=chatprompt.format_messages(
    word1="happy",
    word2="twinning",
    word3="beach",
) 

res=llm.invoke(formatchatprompt)
lines=res.content.split('\n')
for line in lines:
    print(line)

Let's get this caption party started!

Since you've given me three fantastic descriptive words - happy, twinning, and beach - I'll weave them into some fabulous social media captions for you! Here are a few options:

1. "Sunkissed and feeling happy! Nothing beats a beach day with my twins! #beachlife #happyvibes #twinning"
2. "Twinning on the beach, because happiness is catching! #happyplace #beachday #twinning"
3. "Surf's up, and so are our smiles! Happy beach day with my twin, because twinning is winning! #beachlife #happy #twinning"
4. "When in doubt, head to the beach! Happy vibes only with my twin by my side. #beachvibes #happy #twinning"
5. "Twinning and winning on the beach! Happy to share this sun-kissed adventure with my twin. #beachday #happy #twinning"

Feel free to pick the one that makes you go "Ah-ha!" or mix and match to create your own unique caption!


In [4]:
template="Picture yourself as a clever and creative chat assistant that crafts perfect social media captions using three descriptive words: {word1}, {word2}, and {word3}."

prompt=PromptTemplate(input_variables=["word1","word2","word3"],
                    template=template)

formattedprompt=prompt.format(word1="sunny",word2="riverside",word3="friends")
response=llm.invoke(formattedprompt)
lines=response.content.split('\n')
for line in lines:
    print(line)

What a lovely prompt! Here are some creative social media captions using the words "sunny", "riverside", and "friends":

**Happy Moments**

1. "Sunny days with friends by the riverside, life is good!"
2. "Laughter and memories made with friends on a sunny riverside afternoon"

**Adventure Time**

1. "Exploring the great outdoors with friends on a sunny riverside escapade"
2. "Riverside adventures with friends under a sunny sky"

**Relaxation Mode**

1. "Sunny vibes and riverside tranquility with friends by my side"
2. "A sunny day spent with friends by the riverside, recharging and relaxing"

**Special Occasions**

1. "Celebrating [special occasion] with friends on a sunny riverside day"
2. "Surrounded by friends and sunshine on a riverside [special occasion] to remember"

**Inspirational Quotes**

1. "Just like the riverside, life is more beautiful with friends by your side. Sunny days ahead!"
2. "With friends by your side, every day can be a sunny riverside adventure"

I hope you enj